# GermEval 2019 Task 1: Hierarchical Classification of Blurbs

The default project is based on shared task (sort of like competition) held in 2019. It's about multi-label classification of so-called blurbs -- short summaries of books (think about an online bookstore). The language of the blurbs and the labels is German.

The task is multi-label classification: that is, each blurb can be classified to one or many classes. Actually, the classes are hierarchical (e.g., Fantasy -> Urban Fantasy) but we don't really use the hierarchical nature of the classes.

Actually, GermEval 2019 Task 1 has two tasks: task A is about predicting the most general class, and task B is about predicting *all* labels. We only focus on task B.

There are a total of 343 different categories and sub-categories.

The paper describing the task is available [here](https://www.inf.uni-hamburg.de/en/inst/ab/lt/resources/data/germeval-2019-hmc/gest19-1-description.pdf).


What makes the project exciting is that the competition was held in 2019 and many of the teams that competed have published their papers about what they did, and there are also at least some open sourced implementations. The leaderboard of the best submissions is also available in the paper linked above. **Can you outperform the best teams with 2023 technology?**


You can read the papers of  winners here:
 - [Multi-Label Multi-Class Hierarchical Classification using
Convolutional Seq2Seq](https://corpora.linguistik.uni-erlangen.de/data/konvens/proceedings/papers/germeval/Germeval_Task1_paper_2.pdf)
 - [TwistBytes - Hierarchical Classification at GermEval 2019: walking the fine line (of recall and precision)](https://corpora.linguistik.uni-erlangen.de/data/konvens/proceedings/papers/germeval/Germeval_Task1_paper_6.pdf)
 - [Code and paper of the COMTRAVO-DS team](https://github.com/davidsbatista/GermEval-2019-Task_1)


Actually, the papers of all competitiors are available here: https://corpora.linguistik.uni-erlangen.de/data/konvens/proceedings/ (scroll down).


I have implemented the data loading, evaluation and baseline model for you. 

First, let's download the data:

In [ ]:
import sys
import warnings
import torch

In [ ]:
! wget https://www.inf.uni-hamburg.de/en/inst/ab/lt/resources/data/germeval-2019-hmc/germeval2019t1-public-data-final.zip

--2023-05-30 20:24:27--  https://www.inf.uni-hamburg.de/en/inst/ab/lt/resources/data/germeval-2019-hmc/germeval2019t1-public-data-final.zip
Resolving www.inf.uni-hamburg.de (www.inf.uni-hamburg.de)... 134.100.36.5
Connecting to www.inf.uni-hamburg.de (www.inf.uni-hamburg.de)|134.100.36.5|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [application/zip]
Saving to: ‘germeval2019t1-public-data-final.zip’

germeval2019t1-publ     [   <=>              ]  13.27M   669KB/s    in 21s     

2023-05-30 20:24:50 (636 KB/s) - ‘germeval2019t1-public-data-final.zip’ saved [13915224]



In [ ]:
! unzip -u -q germeval2019t1-public-data-final.zip

In [ ]:
! ls -la

total 49368
drwxr-xr-x 1 root root     4096 May 30 20:24 .
drwxr-xr-x 1 root root     4096 May 30 20:01 ..
-rw-r--r-- 1 root root   233276 Jun  7  2019 blurbs_dev_label.txt
-rw-r--r-- 1 root root  1920753 Jun  7  2019 blurbs_dev_nolabel.txt
-rw-r--r-- 1 root root  2630464 Jun  7  2019 blurbs_dev.txt
-rw-r--r-- 1 root root   469047 Aug  9  2019 blurbs_test_label.txt
-rw-r--r-- 1 root root  3787630 Jun  7  2019 blurbs_test_nolabel.txt
-rw-r--r-- 1 root root  5217031 Aug 21  2019 blurbs_test.txt
-rw-r--r-- 1 root root  1630376 Aug  9  2019 blurbs_train_label.txt
-rw-r--r-- 1 root root 18174587 Jun  7  2019 blurbs_train.txt
drwxr-xr-x 2 root root     4096 Sep  2  2019 classification_models
drwxr-xr-x 4 root root     4096 May 26 13:47 .config
-rw-r--r-- 1 root root   251606 Jun  7  2019 description.pdf
drwxr-xr-x 3 root root     4096 Dec  6  2019 evaluation
-rw-r--r-- 1 root root 13915224 Dec  6  2019 germeval2019t1-public-data-final.zip
-rw-rw-rw- 1 root root  2231900 Dec  6  2019 Germeval

The train, dev and test files are in blurbs_{train,dev,test},txt files. They are actually XML files. Let's see:

In [ ]:
!head blurbs_train.txt

<book date="2019-01-04" xml:lang="de">
<title>Die Klinik</title>
<body>Ein Blick hinter die Kulissen eines Krankenhauses vom Autor der Bestseller "Der Medicus" und "Der Medicus von Saragossa". Der Wissenschaftler Adam Silverstone, der kubanische Aristokrat Rafael Meomartino und der Farbige Spurgeon Robinson - sie sind drei grundverschiedene Klinik-Ärzte, die unter der unerbittlichen Aufsicht von Dr. Longwood praktizieren. Eines Tages stirbt eine Patientin, und Dr. Longwood wittert einen Behandlungsfehler. Sofort macht er sich auf die Suche nach einem Schuldigen, dem er die Verantwortung in die Schuhe schieben könnte ...</body>
<copyright>(c) Verlagsgruppe Random House GmbH</copyright>
<categories>
<category>
<topic d="0">Literatur & Unterhaltung</topic>
<topic d="1" label="True">Romane & Erzählungen</topic>
</category>
</categories>


Let's implement data reading using the BeautifulSoup XML library:

In [ ]:
from tqdm.notebook import trange, tqdm

from bs4 import BeautifulSoup
def load_data(filename):
    """
    Loads labels and blurbs of dataset
    """
    data = []
    soup = BeautifulSoup(open(filename, 'rt').read(), "html.parser")
    for book in tqdm(soup.findAll('book')):
      categories = set([])
      book_soup = BeautifulSoup(str(book), "html.parser")
      for t in book_soup.findAll('topic'):
          categories.add(str(t.string))
      data.append((str(book_soup.find("body").string), categories))
    return data

In [ ]:
train_data = load_data("blurbs_train.txt")

  0%|          | 0/14548 [00:00<?, ?it/s]

In [ ]:
train_data[1]

('Die Bedrohungen für Midkemia und Kelewan wollen nicht enden: Obwohl das Konklave der Schatten Leso Varen und seinen Nachtfalken dicht auf den Fersen ist, schmieden sie weiter ihre finsteren Umsturzpläne gegen das Herrscherhaus von Kesh. Zugleich stellt sich heraus, dass von den mysteriösen Talnoy eine bisher ungekannte Gefahr ausgeht: durch ihre magischen Kräfte können die fürchterlichen Dasati ins Reich Midkemia eindringen und alle ins Unheil stürzen …',
 {'Fantasy', 'Heroische Fantasy', 'Literatur & Unterhaltung'})

In [ ]:
dev_data = load_data("blurbs_dev.txt")

  0%|          | 0/2079 [00:00<?, ?it/s]

In [ ]:
dev_data[0]

('Die Konfirmandenzeit wird für Jugendliche besonders dann zu einer nachhaltigen Erfah\xadrung, wenn ihre Eltern Anteil nehmen und sie hilfreich begleiten. Um sie dabei zu unterstützen, bietet diese Broschüre praktische Hinweise zu Formen und Organisation der Konfirmandenzeit in den Gemeinden, Bilder und Berichte zur religiösen und pädagogischen Gestaltung der Konfirmandenarbeit heute sowie Hinweise zur Vorbereitung und Feier der Konfirmation in der Familie. Besonders für die Eltern, für die die Konfirmation ihrer Kinder eine Wiederbegegnung mit Kirche ist, hält dieses Heft zudem Erklärungen zu den Festen und Feiertagen der Kirche, Antworten auf häufig gestellte Fragen und ein kleines Glossar kirchlicher Begriffe bereit. Mit diesem Heft wird die Konfirmandenzeit zu einer Bereicherung auch für die Eltern.',
 {'Gemeindearbeit',
  'Gemeindearbeit mit Kindern & Jugendlichen',
  'Glaube & Ethik',
  'Konfirmation'})

In [ ]:
test_data = load_data("blurbs_test.txt")

  0%|          | 0/4157 [00:00<?, ?it/s]

In [ ]:
test_data[0]

('Ausmalen bringt Freude und entspannt. Dieses meditative Ausmalbuch begleitet Sie mit seinen vielfältigen und wunderschönen Motiven sogar durch das ganze Jahr. Jedem Motiv ist eine inspirierende Affirmation zugeordnet, nach der Sie Ihren Alltag gestalten können. Zudem reduziert tägliches Ausmalen den Stress, erhöht Ihr Wohlbefinden, fördert die Kreativität, verbessert Ihre Konzentration und ist Balsam für die Seelen.',
 {'Entspannung & Meditation',
  'Freizeit & Hobby',
  'Ganzheitliches Bewusstsein',
  'Kreatives',
  'Körper & Seele',
  'Ratgeber'})

In [ ]:
!python -m spacy download de

2023-05-30 20:25:30.153178: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-05-30 20:25:31.120063: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
2023-05-30 20:25:32.399506: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-05-30 20:25:32.399938: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:996] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA n

In [ ]:
import torchtext
from torchtext.data.utils import get_tokenizer
import spacy
tokenizer = get_tokenizer('spacy', language='de')

/usr/local/lib/python3.10/dist-packages/torchtext/data/utils.py:105: UserWarning: Spacy model "de" could not be loaded, trying "de_core_news_sm" instead
  warnings.warn(


In [ ]:
tokenizer("Mit diesem Heft wird die Konfirmandenzeit zu einer Bereicherung auch für die Eltern.")

['Mit',
 'diesem',
 'Heft',
 'wird',
 'die',
 'Konfirmandenzeit',
 'zu',
 'einer',
 'Bereicherung',
 'auch',
 'für',
 'die',
 'Eltern',
 '.']

In [ ]:
from collections import Counter
from torchtext.vocab import vocab

counter = Counter()
for sample in train_data:
    counter.update(tokenizer(sample[0]))
# we'll map all words occurring less than 5 times to <unk>
text_vocab = vocab(counter, min_freq=5, specials=('<unk>', '<BOS>', '<EOS>', '<PAD>'))
text_vocab.set_default_index(text_vocab["<unk>"])

train_labels = []
#labels_counter = Counter()
for sample in train_data:
  for label in sample[1]:
    #labels_counter.update([label])
    if label not in train_labels:
      train_labels.append(label)
# we'll map all words occurring less than 1 times to <unk>
#labels_vocab = vocab(labels_counter, min_freq=1, specials=('<unk>', '<BOS>', '<EOS>', '<PAD>'))
#labels_vocab.set_default_index(text_vocab["<unk>"])

In [ ]:
print("The length of the new vocab is", len(text_vocab))
new_stoi = text_vocab.get_stoi()
print("The index of '<unk>' is", new_stoi['<unk>'])
print("The index of 'Mit' is", new_stoi['Mit'])
new_itos = text_vocab.get_itos()
print("The token at index 2 is", new_itos[2])
print("The token at index 159 is", new_itos[159])

#print("The length of the labels vocab is", len(labels_vocab))
#new_stoi_labels = labels_vocab.get_stoi()
#print("The index of 'Lebenshilfe & Psychologie' is", new_stoi_labels['Lebenshilfe & Psychologie'])
#new_itos_labels = labels_vocab.get_itos()
#print("The token at index 5 is", new_itos_labels[5])


The length of the new vocab is 20864
The index of '<unk>' is 0
The index of 'Mit' is 159
The token at index 2 is <EOS>
The token at index 159 is Mit


In [ ]:
text_transform = lambda x: [text_vocab['<BOS>']] + [text_vocab[token] for token in tokenizer(x)] + [text_vocab['<EOS>']]

#label_transform = lambda x: [labels_vocab[label] for label in x]

# Print out the output of text_transform
print("input to the text_transform:", "Mit diesem Heft wird die Konfirmandenzeit")
print("output of the text_transform:", text_transform("Mit diesem Heft wird die Konfirmandenzeit"))

#print("input to the text_transform:", "Gemeindearbeit, Gemeindearbeit mit Kindern & Jugendlichen,Glaube & Ethik,Konfirmation")
#print("output of the text_transform:", label_transform({'Gemeindearbeit',
#  'Gemeindearbeit mit Kindern & Jugendlichen',
#  'Glaube & Ethik',
#  'Konfirmation'}))

input to the text_transform: Mit diesem Heft wird die Konfirmandenzeit
output of the text_transform: [1, 159, 133, 15323, 187, 7, 2307, 2]


In [ ]:
type(test_data[0][-1])

set

In [ ]:
#mlb = MultiLabelBinarizer()
#train_labels = mlb.fit_transform([set(sample[1]) for sample in train_data])

In [ ]:
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
import numpy as np
from sklearn.preprocessing import MultiLabelBinarizer

def collate_batch(batch):
   label_list, text_list = [], []
   #print(type(train_labels))
   mlb = MultiLabelBinarizer(classes = train_labels)

   for sample in batch:
        _labels = sample[1] # we can have any number of labels here
        _text = sample[0]
        #print(type(_labels))
        #processed_label = torch.tensor(mlb.fit_transform((_labels)))

        #print(processed_label)
        #print(processed_label.shape)
        label_list.append(list(_labels))
        processed_text = torch.tensor(text_transform(_text))
        text_list.append(processed_text)
   res_labels = torch.tensor(mlb.fit_transform(label_list))
   #print(res_labels.shape)
   #print(mlb.classes_)
   return {"labels": res_labels, 
           "tokens": pad_sequence(text_list, batch_first=True, padding_value=text_vocab["<PAD>"]), 
           "lengths": torch.tensor([len(l) for l in text_list])}

train_dataloader = DataLoader(train_data, batch_size=8, shuffle=True, 
                              collate_fn=collate_batch)

test_dataloader = DataLoader(test_data, batch_size=8, shuffle=False, 
                              collate_fn=collate_batch)

In [ ]:
batch = next(iter(train_dataloader))
print(batch)

{'labels': tensor([[1, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 1, 0,  ..., 0, 0, 0],
        ...,
        [0, 1, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0],
        [0, 0, 0,  ..., 0, 0, 0]]), 'tokens': tensor([[    1,    15, 20709,  ...,     3,     3,     3],
        [    1,  3057,    21,  ...,    39,    18,     2],
        [    1, 19019, 19270,  ...,     3,     3,     3],
        ...,
        [    1,   880,  3290,  ...,     3,     3,     3],
        [    1,  4110,   140,  ...,     3,     3,     3],
        [    1,   159,   183,  ...,     3,     3,     3]]), 'lengths': tensor([ 54, 208, 117,  33,  85,  65,  80, 106])}


In [ ]:
batch["tokens"].shape

torch.Size([8, 208])

In [ ]:
[text_vocab.get_itos()[i] for i in batch["tokens"][0]]

['<BOS>',
 'Der',
 'abschließende',
 'Band',
 'des',
 'beispiellosen',
 '<unk>',
 '.',
 'Es',
 'sind',
 'die',
 '<unk>',
 'letzten',
 'Tage',
 '<unk>',
 ',',
 'die',
 'Kempowski',
 'auf',
 '<unk>',
 'eindringliche',
 'Weise',
 'wie',
 'einen',
 'Film',
 'vor',
 'dem',
 'Leser',
 '<unk>',
 'lässt',
 '.',
 'Die',
 '<unk>',
 'Rekonstruktion',
 'aus',
 'Briefen',
 ',',
 'Tagebuchaufzeichnungen',
 ',',
 'Quellen',
 'und',
 'Bildern',
 'ermöglicht',
 'einen',
 'erschütternden',
 'Blick',
 'auf',
 'Leid',
 ',',
 'Propaganda',
 ',',
 'Irrsinn',
 '.',
 '<EOS>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 '<PAD>',
 

In [ ]:
train_dataloader = DataLoader(train_data, batch_size=128, shuffle=True, 
                              collate_fn=collate_batch)

test_dataloader = DataLoader(test_data, batch_size=128, shuffle=False, 
                              collate_fn=collate_batch)

In [ ]:
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
device = 'cpu'
if torch.cuda.is_available():
  device = torch.device('cuda')

print(device)

cuda


In [ ]:
class CnnText(nn.Module):
  
  def __init__(self, num_classes, vocab_size, embedding_dim, hidden_dim, dropout_prob, num_rnn_layers=3):
    super(CnnText, self).__init__()
    self.embed = nn.Embedding(vocab_size, embedding_dim, padding_idx=text_vocab["<PAD>"])
    self.rnn = nn.GRU(embedding_dim, hidden_dim, batch_first=True, bidirectional=False, num_layers=num_rnn_layers)
    #self.conv1 = nn.Conv1d(embedding_dim, 32, kernel_size=3, stride=1)
    #self.conv2 = nn.Conv1d(32, 64, kernel_size=3, stride=1)
    #self.conv3 = nn.Conv1d(64, 64, kernel_size=3, stride=1)
    #self.dropout = nn.Dropout(dropout_prob)
    #self.fc = nn.Linear(64, num_classes)
    self.affine = nn.Linear(hidden_dim, num_classes)
    self.sigm = nn.Sigmoid()
    
  def forward(self, x, x_lengths):
    # Conv1d takes in (batch, channels, seq_len), but raw embedded is (batch, seq_len, channels)
    x = self.embed(x)

    x_post_rnn, _ = self.rnn(x)

    # We take the RNN output not from the last timestep (corresponding to maximum sequence length of the batch
    # but from the actual last time step of the corresponding sequence.

    # Select the last relevant output for each sequence
    x_post_rnn = torch.stack([x_post_rnn[i, length - 1] for i, length in enumerate(x_lengths)])
    #print(x.shape)
    #x = F.relu(self.conv1(x))
    #print(x.shape)
    #x = F.max_pool1d(x, 2)
    #print(x.shape)
    #x = F.relu(self.conv2(x))
    #print(x.shape)
    #x = F.relu(self.conv3(x))
    #print(x.shape)
    #x = F.max_pool1d(x, x.size(2))
    #print(x.shape)
    #x = x.view(-1, 64)
    #print(x.shape)
    #x = self.dropout(x) 
    #logit = self.fc(x)
    logit = self.affine(x_post_rnn)
    return self.sigm(logit)


In [ ]:
model = CnnText(343, len(text_vocab), 100, hidden_dim=256, dropout_prob=0.2).to(device)

In [ ]:
print(model.forward(batch["tokens"].to(device), batch["lengths"].to(device)))

tensor([[0.5086, 0.5248, 0.5082,  ..., 0.5000, 0.5104, 0.5077],
        [0.4956, 0.5244, 0.5176,  ..., 0.5022, 0.4978, 0.5005],
        [0.5012, 0.5158, 0.5054,  ..., 0.5071, 0.5045, 0.4931],
        ...,
        [0.5119, 0.5195, 0.5084,  ..., 0.4995, 0.5057, 0.5091],
        [0.5022, 0.5138, 0.5216,  ..., 0.5075, 0.5027, 0.4998],
        [0.5099, 0.5198, 0.5093,  ..., 0.5103, 0.4976, 0.5029]],
       device='cuda:0', grad_fn=<SigmoidBackward0>)


In [ ]:
from sklearn.metrics import f1_score, recall_score, precision_score 

def calculate_metrics(pred, target, threshold=0.3):
    pred = np.array(pred > threshold, dtype=float)
    #print("predicted after threshold")
    #print(pred)
    return {
            'micro/f1': f1_score(y_true=target, y_pred=pred, average='micro'),
            }

In [ ]:
from tqdm.notebook import tqdm
def train(model, num_epochs, train_iter, test_iter):

  optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

  steps = 0
  best_acc = 0
  last_step = 0
  for epoch in range(1, num_epochs+1):
    print("Epoch %d" % epoch)
    model.train()
    # We wrap the dataloader iterator to tqdm, so that we'll get a nice progress bar
    for batch in tqdm(train_iter, total=len(train_iter)):
      text = batch["tokens"].to(device)
      lengths = batch["lengths"].to(device)
      target = batch["labels"].to(device)
      target = target.float()
      #print("target")
      #print(target.shape)
      #print(target)

      optimizer.zero_grad()
      output = model(text, lengths)
      #print("output")
      #print(output.shape)
      #print(output)

      criterion = nn.BCELoss()

      loss = criterion(output, target)

      #loss = F.nll_loss(output, target)
      #print(loss)

      loss.backward()
      optimizer.step()

      steps += 1

    train_acc = evaluate("train", train_iter, model)                
    dev_acc = evaluate("test", test_iter, model)

def evaluate(dataset_name, data_iter, model):
  
  model.eval()
  batch_losses = []
  #total_corrects, avg_loss = 0, 0
  model_result = []
  targets = []
  with torch.inference_mode():
    for batch in data_iter:
      text = batch["tokens"].to(device)
      lengths = batch["lengths"].to(device)
      target = batch["labels"].to(device)
      target = target.float()

      output = model(text, lengths)

      criterion = nn.BCELoss(reduction='sum')

      #print("Before loss 1")

      loss = criterion(output, target).item()
      batch_losses.append(loss)

      #print("After loss 1")
      #print(loss)

      model_result.extend(output.cpu().numpy())
      targets.extend(target.cpu().numpy())

      fin_output = np.array(model_result)
      #print("final output")
      #print(fin_output)
      fin_targets = np.array(targets)
      #print("final targets")
      #print(fin_targets)
      
      #loss = F.nll_loss(output, target, reduction='sum').item() # sum up batch loss
      #pred = output.argmax(dim=1, keepdim=True) # get the index of the max log-probability
      #correct = pred.eq(target.view_as(pred)).sum().item()
      
      #avg_loss += loss
      
      #total_corrects += correct
    result = calculate_metrics(fin_output, fin_targets)
    print("Evaluation on {} - micro f1: {:.3f} ".format(dataset_name, 
                                      result['micro/f1']))
                                      #result['macro/f1'],
                                      #result['samples/f1']))
    loss_value = np.mean(batch_losses)
    print("Evaluation on {} - loss:{:.3f}".format(dataset_name, loss_value))
    #size = len(data_iter.dataset)
    #avg_loss /= size
    #accuracy = 100.0 * total_corrects/size
    #print('  Evaluation on {} - loss: {:.3f}  acc: {:.2f}%({}/{})'.format(dataset_name,
    #                                                                  avg_loss, 
    #                                                                  accuracy, 
    #                                                                  total_corrects, 
    #                                                                  size))
    return result['micro/f1']                

In [ ]:
model = CnnText(343, len(text_vocab), 100, hidden_dim=256, dropout_prob=0.2).to(device)
train(model, 8, train_dataloader, test_dataloader)

Epoch 1


  0%|          | 0/114 [00:00<?, ?it/s]

Evaluation on train - micro f1: 0.262 
Evaluation on train - loss:1613.121
Evaluation on test - micro f1: 0.254 
Evaluation on test - loss:1619.785
Epoch 2


  0%|          | 0/114 [00:00<?, ?it/s]

Evaluation on train - micro f1: 0.262 
Evaluation on train - loss:1614.248
Evaluation on test - micro f1: 0.254 
Evaluation on test - loss:1620.007
Epoch 3


...

  0%|          | 0/114 [00:00<?, ?it/s]

Evaluation on train - micro f1: 0.262 
Evaluation on train - loss:1610.658
Evaluation on test - micro f1: 0.254 
Evaluation on test - loss:1619.491
Epoch 7


  0%|          | 0/114 [00:00<?, ?it/s]

Evaluation on train - micro f1: 0.262 
Evaluation on train - loss:1611.401
Evaluation on test - micro f1: 0.254 
Evaluation on test - loss:1619.110
Epoch 8


  0%|          | 0/114 [00:00<?, ?it/s]